<a href="https://colab.research.google.com/github/sebastianSbg/AML/blob/main/project_1/AML_project_1_birte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import pandas as pd
import pickle
import numpy as np
import os

from google.colab import files
import io
import time

In [54]:
# mounting google drive  -> choose files from folder on the left vertical taskbar
from google.colab import drive
drive.mount('/content/drive')

# specify base directory for files
baseDir = '/content/drive/My Drive/ETHZ/AML/Project_1/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##  reading data

In [55]:
D_X_df = pd.read_csv(baseDir + 'X_train.csv', index_col = 'id')
D_y_df = pd.read_csv(baseDir + 'y_train.csv', index_col = 'id')
D_test_df = pd.read_csv(baseDir + 'X_test.csv', index_col = 'id')

D_X_df.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,...,x792,x793,x794,x795,x796,x797,x798,x799,x800,x801,x802,x803,x804,x805,x806,x807,x808,x809,x810,x811,x812,x813,x814,x815,x816,x817,x818,x819,x820,x821,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,10.891876,832442.812375,20585.544083,1028.369495,1.163780e+06,9.199135,597900.477629,NaN,1.144294e+06,785176.201298,38042.008654,106.644316,67475.900839,944696.306555,895.088214,10.819457,155995.454645,10.114602,98.776154,90.200020,107.176656,1.232961e+06,2538.045240,1.038292e+06,10.445353,1530.001316,10403.207060,101.974225,9902.072478,10.675578,1.965179,108.095109,10.491893,985.743949,139494.789412,5555.414472,10079.665710,NaN,1199.950457,14709.216512,...,2.586985,107835.473217,1.050203e+06,961.000963,102.726565,11230.077183,1132.878438,10955.849718,2.358928,1.079014e+06,10.271036,309702.424398,1074.952720,10083.165942,106208.115341,102.072379,119.507651,NaN,1077.836958,NaN,87766.029819,1857.070822,96.997544,NaN,1011.959739,89.184535,107577.830266,2.139276,-3163.219791,1057.888075,1.024198e+06,-855.549602,12176.073427,10.647729,10.916371,1220.065443,8.566724,1.036263e+06,85338.558539,103088.664210
1.0,11.512994,832442.898114,NaN,1012.624877,1.028911e+06,10.906408,597900.458612,8127.016078,1.099166e+06,785176.258299,50949.420639,NaN,93179.779900,926043.432843,1159.073327,10.403125,NaN,10.365511,104.569049,88.722820,102.703415,NaN,2033.004526,8.321971e+05,10.503675,2119.097861,10700.279888,101.354911,10414.075263,10.860626,1.716548,102.045514,10.073451,880.965682,NaN,4784.223551,10968.228448,NaN,937.225484,12616.740072,...,1.796449,104811.157714,1.091030e+06,1052.815245,82.458489,10487.069231,695.406075,10985.255804,1.886463,1.072502e+06,8.178066,557053.464687,1060.352954,10458.065277,108798.115287,100.207625,NaN,95559.700238,1028.505709,212339.931756,99387.412802,1818.087105,101.159059,1073.745683,1017.273256,NaN,NaN,1.746126,-2661.694233,1025.245447,1.086806e+06,-787.397942,10493.095660,10.586492,9.463962,917.094909,10.231822,1.007163e+06,95695.020645,105161.109422
2.0,11.052185,832442.896307,20585.512844,1003.953827,9.231756e+05,9.212979,597900.426764,10738.092422,1.027863e+06,785176.223468,47259.321206,103.365571,69377.195188,828900.306175,NaN,10.667846,155995.367237,10.437764,94.987987,101.695502,100.034049,NaN,2138.054129,6.792212e+05,10.967175,1926.041068,10633.374493,107.429192,12163.033117,10.581398,2.340156,101.247931,10.782089,NaN,139494.776583,4317.864410,10628.203050,10381.020335,977.000200,13070.565105,...,2.932762,101814.683255,1.001865e+06,NaN,96.292749,11122.087230,862.889649,10308.209427,2.406808,1.048036e+06,NaN,396496.091177,1004.185813,10249.487036,NaN,106.147877,94.671834,91827.987615,1061.749457,212339.935348,108348.187401,1895.067520,NaN,1015.708474,903.520905,100.032696,108205.355183,2.421048,-2671.968203,1039.024306,1.018533e+06,-906.997242,10959.516944,10.769287,10.342160,637.027802,10.705461,1.019955e+06,80253.299882,104177.051666
3.0,11.642076,NaN,NaN,1004.672084,9.459461e+05,9.553420,597900.450367,13524.096973,1.168144e+06,785176.254867,53507.052907,NaN,85953.175663,903459.376783,NaN,10.310333,155995.459579,10.916521,116.303369,125.945559,100.950876,8.504439e+05,2618.086812,NaN,11.003482,2402.018255,10786.767836,103.732506,17640.001069,10.872748,2.424319,106.297510,10.309167,1028.878602,139494.765626,5728.033993,10139.014032,10450.816795,1074.215843,20564.616684,...,2.957477,107142.205872,NaN,980.714878,98.959512,13292.071797,NaN,10864.961338,2.646011,1.043134e+06,11.016208,496861.936593,1058.957153,10797.087584,109698.613381,108.188333,98.048619,87665.325337,1060.987085,212339.966651,95656.503335,2390.011297,NaN,NaN,836.111641,86.555080,105872.407925,2.618742,-4593.300507,1095.505032,1.047017e+06,-1011.742516,16845.309819,10.483830,10.594941,1114.069590,10.32106

## Imputing data

In [56]:
# Method 1: imputing nan data and replace by median
D_X_df = D_X_df.fillna(D_X_df.median())
D_test_df = D_test_df.fillna(D_test_df.median())

# Method 2 (result same as 1): imputing nan data and replace by median
from sklearn.impute import SimpleImputer

imp_mean = SimpleImputer(missing_values=np.nan, strategy='median').fit(D_X_df)
D_X_df = pd.DataFrame(imp_mean.transform(D_X_df), index = D_X_df.index)
imp_mean = SimpleImputer(missing_values=np.nan, strategy='median').fit(D_test_df)
D_test_df = pd.DataFrame(imp_mean.transform(D_test_df), index = D_test_df.index)

# visualizing
D_X_df.head()
D_test_df.head(20)

# Method 3: advanced imputing
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer

# imp_mean = IterativeImputer(random_state=0).fit(D_X_df)
# D_X_df = pd.DataFrame(imp_mean.transform(D_X_df),index=D_X_df.index)

# imp_mean = IterativeImputer(random_state=0).fit(D_test_df)
# D_test_df = pd.DataFrame(imp_mean.transform(D_test_df),index=D_test_df.index)

# D_test_df.head(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,9.101943,832442.839400,20585.538590,1049.164447,1.005104e+06,10.417525,597900.393691,10791.058148,9.829612e+05,785176.180458,50511.594745,102.435963,86093.362703,1.004573e+06,896.242127,10.375483,155995.366062,10.508879,109.800880,100.993456,108.804989,8.781876e+05,2326.063359,1.138527e+06,10.297648,2011.015908,10244.853851,104.527727,11382.031248,10.028024,2.079459,106.041150,10.776713,1038.617368,139494.768281,5825.790249,10997.336251,10400.640740,1116.394788,13501.871818,...,2.368853,104526.315367,1.081410e+06,1049.543694,97.849732,11808.554313,1000.290615,10579.672076,2.428940,1.059232e+06,10.465314,421831.383476,1015.262615,10815.643746,104879.207337,104.011715,100.286775,90475.839928,1067.281801,212339.928915,103810.690956,2352.069763,94.000643,1002.249497,993.353967,99.862166,106258.939870,2.264941,-2694.807672,1036.171909,1.049199e+06,-1002.286934,11388.219873,10.538011,9.070287,1122.095390,9.590537,1.006411e+06,99818.695142,106668.615874
1.0,11.722077,832442.826314,20585.509289,1039.097880,9.836399e+05,11.185000,597900.474419,10321.030961,9.998527e+05,785176.200371,41407.903610,101.088941,76637.435738,1.004495e+06,1035.185204,10.659824,155995.462172,10.947247,100.575341,101.772209,82.815109,1.197197e+06,2633.000136,8.853794e+05,10.276833,2053.047024,10303.168389,105.831421,12732.043012,10.505525,2.112429,102.880796,10.425907,859.407229,139494.732463,5163.481295,10198.152721,10066.482625,914.593856,19057.813333,...,2.544295,107303.807199,1.070458e+06,1042.129135,100.869251,13067.032089,1030.473888,10214.760012,2.506911,1.082130e+06,8.724603,385330.203143,1076.546084,10741.440845,104208.340132,100.717323,107.324110,114387.912850,1042.580507,212339.960354,106239.952419,1342.021781,109.551775,1043.778938,1149.864366,105.839356,100456.933966,2.304030,-4410.495371,1062.123154,1.063850e+06,-777.155703,15454.964631,10.438084,10.573977,830.040750,11.283446,1.064455e+06,100761.264268,105023.965025
2.0,11.200277,832442.820359,20585.511136,1081.926822,1.059913e+06,10.073223,597900.432849,9394.048739,8.439144e+05,785176.185410,46674.483452,104.817714,79959.241357,1.131082e+06,987.229922,10.155268,155995.378394,10.058355,104.651529,107.515927,98.427233,1.149049e+06,2325.042316,8.593389e+05,10.843412,1828.017538,10904.570958,108.128202,14776.099270,10.430659,2.015715,101.922523,10.257955,881.208979,139494.816274,6018.141886,10024.516679,10140.733864,954.686726,16564.196065,...,2.444435,106631.044769,1.073753e+06,1004.506025,76.162439,11597.007704,1054.414589,10174.696701,2.265824,1.063598e+06,10.401901,426875.862408,1008.289838,10761.782595,103792.281658,104.607906,95.147810,111215.554821,1036.620439,212339.951432,88457.927330,1746.038436,96.258134,1089.959540,1174.541795,97.029949,106838.188610,2.125303,-3653.593571,1010.660414,1.033983e+06,-866.209778,13616.308487,10.123607,8.273391,926.072523,9.277080,1.007427e+06,106440.456728,103405.273232
3.0,9.668873,832442.820901,20585.528528,1063.771791,1.023284e+06,9.635705,597900.426803,10640.030819,1.018818e+06,785176.263405,47386.653465,104.817714,90440.005438,9.759018e+05,980.339017,10.630591,155995.463081,10.554582,106.310312,94.185668,98.934510,1.120927e+06,3205.086592,1.006123e+06,10.911877,2229.046705,10283.402609,102.019870,15253.023408,10.867628,2.079459,104.912704,10.276334,1243.431251,139494.765587,4791.785060,10853.086751,10725.048648,933.370128,17741.321420,...,2.488275,107230.416352,1.066114e+06,1122.508118,95.633283,14557.055407,996.662291,10498.083192,2.302926,1.098057e+06,10.933728,450538.394923,1070.477176,10401.406184,105967.024852,103.385839,102.425165,81237.923270,1074.308433,212339.956291,106239.827794,1826.000

## Converting to numpy array

In [57]:
# defining numpy arrays 
D_X = np.array(D_X_df)
D_y = np.array(D_y_df)
D_test = np.array(D_test_df)

## Remove outliers in the training data set

In [58]:
from sklearn.ensemble import IsolationForest

# Data dimensions before outlier removal
print(D_X.shape, D_y.shape)

# Identify outliers
iso = IsolationForest(contamination=0.05)
yhat = iso.fit_predict(D_X)

# select all rows that are not outliers
mask = yhat != -1
D_X, D_y = D_X[mask, :], D_y[mask]

# Dimensions after outlier removal
print(D_X.shape, D_y.shape)

(1212, 832) (1212, 1)
(1151, 832) (1151, 1)


# standardizing data - normalizing data around zero & unit std

In [59]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()

scaler.fit_transform(D_X)
D_X_stand = scaler.transform(D_X)
D_y = np.ravel(D_y)
D_test_stand = scaler.transform(D_test)

pd.DataFrame(D_test_stand).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831
0,-0.994800,-0.725982,0.499252,-0.002179,0.043372,0.355948,-1.343956,0.298157,-0.170455,-1.626177,1.048649,-0.997863,0.769292,0.030878,-1.167164,-0.685124,-1.692964,-0.154002,1.045921,0.058784,1.045209,-1.284601,-0.184074,1.503409,-0.888124,0.397650,-0.958731,-0.200929,-0.703775,-1.848317,-0.021022,0.409538,0.890345,0.400764,-0.109339,1.201281,1.909078,-0.368123,1.258650,-0.986766,...,-1.063191,-0.175171,1.180148,0.519927,-0.218698,-0.114028,0.017516,0.398805,0.351756,0.330166,0.510051,-0.086847,-1.275818,1.149302,-0.071627,-0.341077,0.092784,-1.059553,0.699217,-0.681047,0.351167,1.460015,-0.667072,-1.759413,-0.050954,-0.093206,0.472028,0.332369,1.270673,-0.494230,-0.011517,-0.856227,-0.900897,-0.058223,-1.054184,0.309135,-0.417634,-1.634346,-0.009377,0.676164
1,1.829000,-1.207952,-0.548720,-0.371824,-0.187345,1.177856,1.666429,-0.014000,0.001306,-0.903823,-0.664771,-1.501648,-0.240210,0.030018,0.351309,0.354393,1.773291,1.499537,0.040948,0.142811,-1.846546,2.071594,0.501922,-1.182736,-0.963874,0.546214,-0.740679,0.273323,-0.047880,-0.100538,0.195688,-0.753916,-0.395501,-1.550786,-1.424844,0.308121,-1.139977,-1.612391,-0.871592,0.954571,...,-0.050528,0.845883,0.768954,0.440530,0.094990,0.634841,0.328486,-0.979051,0.808337,1.161478,-1.301483,-0.662374,0.936746,0.879244,-0.315961,-1.525468,0.866450,1.536825,-0.218648,0.453925,0.606314,-1.382033,1.045189,-0.224898,1.610777,0.553819,-1.637594,0.565720,-1.200343,0.474672,0.521038,0.636928,0.903611,-0.426249,0.543799,-1.091258,1.304913,0.529194,0.092696,0.052041
2,1.266641,-1.427288,-0.482680,1.200856,0.632524,-0.012774,0.116251,-0.629633,-1.584347,-1.446561,0.326460,-0.107091,0.114421,1.414020,-0.172782,-1.490201,-1.248199,-1.853392,0.484983,0.762553,-0.109469,1.565044,-0.186356,-1.459052,1.098026,-0.249178,1.508111,1.108836,0.945213,-0.374566,-0.440001,-1.106696,-1.011109,-1.313371,1.653345,1.460678,-1.802436,-1.335910,-0.448366,0.083260,...,-0.626929,0.598564,0.892663,0.037650,-2.471719,-0.239911,0.575141,-1.130325,-0.603414,0.488667,0.444057,-0.007309,-1.527561,0.953277,-0.467492,-0.126736,-0.472181,1.192370,-0.440116,0.131849,-1.261341,-0.245221,-0.418510,1.481478,1.872786,-0.399791,0.682644,-0.501230,-0.110217,-1.446714,-0.564634,0.046288,0.087757,-1.584458,-1.901052,-0.630789,-0.736578,-1.596459,0.707709,-0.562231
3,-0.383801,-1.407348,0.139360,0.534203,0.238800,-0.481323,-0.109219,0.197856,0.194153,1.382753,0.460499,-0.107091,1.233334,-0.282594,-0.248091,0.247519,1.806105,0.018391,0.665680,-0.675771,-0.053027,1.269182,1.780522,0.098467,1.347182,1.168304,-0.814588,-1.113227,1.176923,1.224854,-0.021022,-0.005889,-0.943744,2.631132,-0.208296,-0.193133,1.358735,0.839840,-0.673387,0.494567,...,-0.373882,0.818903,0.605854,1.301253,-0.448958,1.521494,-0.019865,0.090737,-0.386152,1.739671,0.997522,0.365786,0.717635,-0.358296,0.324564,-0.566090,0.327873,-2.062608,0.960316,0.307246,0.606300,-0.020227,0.183328,-0.890597,1.217357,-1.436338,-0.036909,0.227990,-0.467914,0.429184,0.532356,-0.021060,0.522628,0.748255,-1.617149,0.044972,-0.200532,0.980384,-0.009377,-1.652615
4,0.328675,0.468958,-1.488687,-1.318349,-0.660569,0.620351,-1.114889,0.080990,0.253101,0.968857,0.993978,-0.107091,-0.029672,0.033216,-0.206936,-1.599768,-0.713315,-1.932710,-0.101735,-1.095944,0.071065,1.407137,-0.617785,1.084928,-1.637033,0.786555,1.690707,1.328258,0.341264,-0.301958,1.159781,-0.954324,-0.307465,1.294220,1.406257,0.139921,1.651441,0.327624,0.534586,0.330341,...,0.470126,0.551899,0.560428,2.257248,-0.117286,-0.038168,1.372410,0.983657,0.602072,1.782842,0.646667,-0.530515,-0.975438,-1.381386,-0.622008,0.163352,0.766413,1.078283,0.852228,-0.930360,0.283794,-0.138284,0.817582,-0.696663,-0.701904,1.534572,0.515290,-0.375265,-0.159098,1.5

# K-fold  - perhaps make it better by using shuffled classes

In [60]:
from sklearn.model_selection import KFold

kf = KFold(10,shuffle=True,random_state=13)

ind = np.arange(D_X.shape[0])

ind_splits = list(kf.split(ind))

## RBF regression with predict

In [61]:
# getting time when starting training
import datetime
now = datetime.datetime.now()
unixTime = round(now.timestamp())

In [62]:
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

bestModel = None
score_best = float('-inf')
tempR2scores = []
tempModels = []

for idx, (train_index, val_index) in enumerate(ind_splits):

  D_X_train = D_X_stand[train_index,:]
  D_y_train = D_y[train_index]

  D_X_val = D_X_stand[val_index,:]
  D_y_val = D_y[val_index]

  #svr prediction
  svc = svm.SVR(kernel='rbf', verbose=True)
  model = svc.fit(D_X_train,D_y_train)
  score_m = model.score(D_X_val, D_y_val)

  y_pred = model.predict(D_X_val)

  # Rsquared score minimize
  # r2 = 0.5 + 0.5 * np.maximum(0, metrics.r2_score(D_y_val, y_pred))
  r2 = r2_score(D_y_val, y_pred)

  tempModels.append(model)
  tempR2scores.append(r2)
  
  # shouldn't choose lowest r2 score since this indicates overfitting, rather mean -> change this in the future
  if r2 > score_best:
    score_best = r2
    bestModel = model
    print(f'\nBest model has a score of {score_best}')

[LibSVM]
Best model has a score of 0.241593406074589
[LibSVM]
Best model has a score of 0.2777365579825699
[LibSVM]
Best model has a score of 0.30855579133113
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

## predicting

In [52]:
result_r = bestModel.predict(D_test_stand)
result_r = np.transpose(np.array(result_r))

print(result_r.shape)

(776,)


## np arrays back to pandas dataframe TO DO: convert id to integer

In [40]:
D_test_df.index
D_y_df.columns

df_r = pd.DataFrame(result_r,index=D_test_df.index)
df_out = pd.concat([df_r], axis=1, sort=False)
df_out.columns = D_y_df.columns
df_out.head()

,y
id,
0.0,69.781573
1.0,69.652969
2.0,70.278185
3.0,68.985180
4.0,69.109432


## saving dataframe TO DO: add time feature

In [41]:
name = f'{baseDir}y_test_{unixTime}.csv'
print(f'File was saved under {name}')
df_out.to_csv(name, index=True, header = True, float_format='%.3f') #, compression='zip')

File was saved under /content/drive/My Drive/ETHZ/AML/Project_1/y_test_1601993216.csv
